## 1. Creating Hashed Passwords for Authentication:

* Everytime when a new user is added to the app, we need to get the user_id (usually email) and their password from them.
* Then we need to create a `Hash` of the password and add it to the `authenticator.yml` file in order for the user to be able to log-in to the app.

In [4]:
from streamlit_authenticator.utilities.hasher import Hasher

hashed_passwords = Hasher(['abc123']).generate()
print(hashed_passwords) 

['$2b$12$tdKt5p7nfhQgcDG30FldqeTZLUio5P.vAqKXSqiUHxURaei7Npkq2']


In [5]:
hashed_passwords = Hasher(['blackpearl']).generate()
print(hashed_passwords) 

['$2b$12$N9w9IGLBfLv1tne8lzalUOm2hH4ytnvbTgQ.vs0jV6EbKt9x2QPr6']


## 2. Testing the connection with Snowflake Data warehouse:

In [32]:
import snowflake.connector
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [34]:
con = snowflake.connector.connect(
    user='hariharan',
    password= os.getenv('SNOWSQL_PWD'),
    account=os.getenv('SNOWSQL_ACCOUNT'),
    warehouse = os.getenv('SNOWSQL_WAREHOUSE'),
)

In [35]:
# Show all the databses present in the database
cur = con.cursor()
cur.execute("SHOW DATABASES")
result = cur.fetchall()
for row in result:
    schema_name = row[1]
    print(schema_name)

FOOD_DELIVERY
SNOWFLAKE
SNOWFLAKE_SAMPLE_DATA


In [36]:
# Show all the schemas present in the database
cur = con.cursor()
cur.execute("SHOW SCHEMAS IN FOOD_DELIVERY")
result = cur.fetchall()
for row in result:
    schema_name = row[1]
    print(schema_name)

CORE
INFORMATION_SCHEMA
PUBLIC


In [37]:
# Show all the tables present in the schema CORE
cur.execute("SHOW TABLES IN SCHEMA FOOD_DELIVERY.CORE")
result = cur.fetchall()
for row in result:
    table_name = row[1]
    print(table_name)

MENU_ITEMS
ORDERS
ORDER_DETAILS
PAYMENTS
RESTAURANTS
REVIEWS
USERS


In [38]:
# creating cursor object
cur = con.cursor()

# Execute a statement that will generate a result set.
sql = "SELECT * FROM FOOD_DELIVERY.CORE.ORDERS"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
df = cur.fetch_pandas_all()

print(df.shape)
df.head()

(73748, 7)


,ORDER_ID,USER_ID,ORDER_TIME,DELIVERY_ADDRESS,ORDER_STATUS,RESTAURANT_ID,TOTAL_AMOUNT
0,716d868b-2b5a-4bee-bc2c-ee262c81588f,d36e1a86-6e33-434a-b9c7-3f5c30753402,2024-06-01 11:53:19,"9198 Gabriela Green\nEast Marcton, DC 16873",Delivered,R18329211,152.85
1,0f41a3a7-954a-4e24-ad84-dba22e8dd154,1ae81af6-6b78-4695-8996-442e9e40ad3c,2024-06-01 08:11:05,"366 Byrd Hills\nNew Robert, WI 24455",Delivered,R22670500,191.50
2,fde175ad-d2ea-4901-a5fd-5c28f44e7382,2c1492fd-c993-4d00-9086-dc105c821bae,2024-06-01 21:46:46,"71211 Gregory Track\nGreenestad, OH 72514",Delivered,R66375744,35.88
3,99e370cd-f55e-4793-b0e6-0c41d8fd9287,523d1f1a-9edd-4811-9472-f65a52d51f15,2024-06-01 09:04:39,"070 Steven Heights\nRoachville, PW 12962",Delivered,R42644875,149.57
4,a88b8bf1-9fdf-4c8e-87ab-ac3a390d05e0,a6c31749-eb2a-40f3-aa65-49ead5cfdc3a,2024-06-01 21:05:23,"9382 Alyssa Branch\nShellyfurt, VI 11466",Delivered,R23024716,129.32


## 3. Testing the connection with Databricks Data warehouse:

In [53]:
from dotenv import load_dotenv
import os 
import pandas as pd 
from databricks import sql 

load_dotenv()

True

In [54]:
# Creating a connection to databricks
connection = sql.connect(server_hostname=os.getenv("DATABRICKS_SERVER_HOST_NAME"),
                         http_path=os.getenv("DATABRICKS_HTTP_PATH"),
                         access_token=os.getenv("DATABRICKS_ACCESS_TOKEN")
                         )


In [56]:
df = pd.read_sql("SELECT * FROM hive_metastore.online_food_business.menu_items", connection)
print(df.shape)
df.head()

/var/folders/3g/7x619qt169344pmm32qd8wgc0000gq/T/ipykernel_8801/705794241.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM hive_metastore.online_food_business.menu_items", connection)


(3000, 4)


,menu_item_id,restaurant_id,item_name,price
0,R32379007_1,R32379007,Tacos,10.20
1,R32379007_2,R32379007,Burritos,13.93
2,R32379007_3,R32379007,Enchiladas,14.15
3,R32379007_4,R32379007,Quesadillas,10.47
4,R32379007_5,R32379007,Nachos,10.80
